<a href="https://colab.research.google.com/github/surkhanjay-ops/YOLACT-Instance-Segmentation/blob/main/YOLACT1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/dbolya/yolact.git
!cd yolact

Cloning into 'yolact'...
remote: Enumerating objects: 2942, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 2942 (delta 2), reused 2 (delta 2), pack-reused 2938 (from 2)
Receiving objects: 100% (2942/2942), 21.21 MiB | 26.78 MiB/s, done.
Resolving deltas: 100% (2004/2004), done.


In [3]:
# Install PyTorch and TorchVision with CUDA support.
# The command below installs the latest stable version, which is much higher than 1.0.1.
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


### Verify the installation

In [4]:
import torch
import torchvision

print(f"PyTorch version: {torch.__version__}")
print(f"TorchVision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.9.0+cu126
TorchVision version: 0.24.0+cu126
CUDA available: True


In [5]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib

In [6]:
# Make sure we're in the top folder
%cd /content

# Clone the repo
!git clone https://github.com/dbolya/yolact.git

%cd yolact

/content
fatal: destination path 'yolact' already exists and is not an empty directory.
/content/yolact


After installing the requirements, you might want to explore the project structure or run some of the provided scripts (e.g., for training or testing). For example, you can list the contents of the directory:


In [7]:
!ls

backbone.py	 eval.py   __pycache__	     scripts   yolact
CHANGELOG.md	 external  README.md	     train.py  yolact.py
data		 layers    results	     utils
environment.yml  LICENSE   run_coco_eval.py  web


In [8]:
!pwd

/content/yolact


In [9]:
!sh data/scripts/COCO.sh

navigating to ./data/ ...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.0G  100 18.0G    0     0  54.9M      0  0:05:35  0:05:35 --:--:-- 62.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  777M  100  777M    0     0  56.4M      0  0:00:13  0:00:13 --:--:-- 60.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241M  100  241M    0     0  47.3M      0  0:00:05  0:00:05 --:--:-- 49.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  241M  100  241M    0     0  52.7M      0  0:00:04  0:00:04 --:--:-- 55.6M
Finished downloading. Now 

In [10]:
!pwd

/content/yolact


In [11]:
!sh data/scripts/COCO_test.sh

navigating to ./data/ ...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6339M  100 6339M    0     0  53.2M      0  0:01:59  0:01:59 --:--:-- 57.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1117k  100 1117k    0     0  1444k      0 --:--:-- --:--:-- --:--:-- 1443k
Finished downloading. Now extracting ...
Extracting train images ...
Extracting info ...
Removing zip files ...
Completed in  189  seconds


In [12]:
!pwd

/content/yolact


In [13]:
# ========================================
# 3. Download Pre-trained Weights
# ========================================
# Create weights directory
!mkdir -p weights

# Download YOLACT base model (ResNet101-FPN, 29.8 mAP)
!wget https://huggingface.co/dbolya/yolact-base/resolve/main/yolact_base_54_800000.pth?download=true -O weights/yolact_base_54_800000.pth


--2026-01-21 05:03:52--  https://huggingface.co/dbolya/yolact-base/resolve/main/yolact_base_54_800000.pth?download=true
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.114, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/68bf6781b93b1747ce962648/191dd7a7b35f7727eab3512ebde7a7670e873acfd092aeb0cb6a095c5cfabe9d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27yolact_base_54_800000.pth%3B+filename%3D%22yolact_base_54_800000.pth%22%3B&Expires=1768975433&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4OTc1NDMzfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjhiZjY3ODFiOTNiMTc0N2NlOTYyNjQ4LzE5MWRkN2E3YjM1Zjc3MjdlYWIzNTEyZWJkZTdhNzY3MGU4NzNhY2ZkMDkyYWViMGNiNmEwOTVjNWNmYWJlOWRcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=hH0--Qjp8bAnlWd

In [14]:
#YOLACT ResNet50 (28.2 mAP, fastest)
!wget https://huggingface.co/dbolya/yolact-resnet50/resolve/main/yolact_resnet50_54_800000.pth?download=true -O weights/yolact_resnet50_54_800000.pth

--2026-01-21 05:04:05--  https://huggingface.co/dbolya/yolact-resnet50/resolve/main/yolact_resnet50_54_800000.pth?download=true
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/68bf6ccf8efa8396976efd76/52269c4b4fb9e83aecfd1bf8db2dfad0f675a9577507f621a33644ea244dece5?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27yolact_resnet50_54_800000.pth%3B+filename%3D%22yolact_resnet50_54_800000.pth%22%3B&Expires=1768975445&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY4OTc1NDQ1fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjhiZjZjY2Y4ZWZhODM5Njk3NmVmZDc2LzUyMjY5YzRiNGZiOWU4M2FlY2ZkMWJmOGRiMmRmYWQwZjY3NWE5NTc3NTA3ZjYyMWEzMzY0NGVhMjQ0ZGVjZTVcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature

In [15]:
#Make a new directory for the test images
!mkdir /content/test_images

# Move to the new directory
%cd /content/test_images

# Make a list of URLs from the COCO dataset
get_imgs = ['http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg',
            'http://farm3.staticflickr.com/2835/9520716824_51220b13d0_z.jpg',
            'http://farm3.staticflickr.com/2817/10105052924_0c8d79e62c_z.jpg',
            'https://farm3.staticflickr.com/2637/3679968895_37a79e55b3_z.jpg',
            'https://farm8.staticflickr.com/7137/7705349298_6bea29cf1c_z.jpg',
            'http://farm5.staticflickr.com/4105/4981945644_3f162314ed_z.jpg',
            'http://farm1.staticflickr.com/40/101757885_a8fc40e273_z.jpg',
            'http://farm3.staticflickr.com/2654/3897535041_fa8dd5570b_z.jpg']

# Download all the images
for img_url in get_imgs:
  !wget {img_url} -nc

# List the contents of the folder. Should have images.
!pwd
!ls

/content/test_images
--2026-01-21 05:04:13--  http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Resolving farm8.staticflickr.com (farm8.staticflickr.com)... 52.85.103.59, 2600:9000:26ce:d600:0:5a51:64c9:c681, 2600:9000:26ce:ac00:0:5a51:64c9:c681, ...
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|52.85.103.59|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg [following]
--2026-01-21 05:04:13--  https://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg
Connecting to farm8.staticflickr.com (farm8.staticflickr.com)|52.85.103.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘6946559805_7edf757660_z.jpg’

6946559805_7edf7576     [ <=>                ]  52.80K  --.-KB/s    in 0.009s  

2026-01-21 05:04:13 (5.69 MB/s) - ‘6946559805_7edf757660_z.jpg’ saved [54067]

--2026-01-21 05:04:14--  http:

In [16]:
!pwd

/content/test_images


In [17]:
%cd /content/yolact

/content/yolact


In [18]:
!ls

backbone.py	 eval.py   __pycache__	     scripts   weights
CHANGELOG.md	 external  README.md	     train.py  yolact
data		 layers    results	     utils     yolact.py
environment.yml  LICENSE   run_coco_eval.py  web


In [19]:
!pwd

/content/yolact


In [20]:
# Quantitatively evaluate a trained model on the entire validation set. Make sure you have COCO downloaded as above.
# This should get 29.92 validation mask mAP last time I checked.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth

Config not specified. Parsed yolact_base_config from the file name.

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1275: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)
loading annotations into memory...
Done (t=0.57s)
creating index...
index created!
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'downsample_l

In [21]:
# Output a COCOEval json to submit to the website or to use the run_coco_eval.py script.
# This command will create './results/bbox_detections.json' and './results/mask_detections.json' for detection and instance segmentation respectively.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --output_coco_json

# You can run COCOEval on the files created in the previous command. The performance should match my implementation in eval.py.
!python run_coco_eval.py

# To output a coco json file for test-dev, make sure you have test-dev downloaded from above and go
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --output_coco_json --dataset=coco2017_testdev_dataset

Config not specified. Parsed yolact_base_config from the file name.

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1275: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)
loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'downsample_l

In [22]:
# Display qualitative results on COCO. From here on I'll use a confidence threshold of 0.15.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.15 --top_k=15 --display

Streaming output truncated to the last 5000 lines.
Avg FPS: 21.7251
Figure(640x480)
Avg FPS: 21.7243
Figure(640x480)
Avg FPS: 21.7241
Figure(640x480)
Avg FPS: 21.7247
Figure(640x480)
Avg FPS: 21.7260
Figure(640x480)
Avg FPS: 21.7267
Figure(640x480)
Avg FPS: 21.7285
Figure(640x480)
Avg FPS: 21.7288
Figure(640x480)
Avg FPS: 21.7295
Figure(640x480)
Avg FPS: 21.7292
Figure(640x480)
Avg FPS: 21.7294
Figure(640x480)
Avg FPS: 21.7290
Figure(640x480)
Avg FPS: 21.7300
Figure(640x480)
Avg FPS: 21.7302
Figure(640x480)
Avg FPS: 21.7306
Figure(640x480)
Avg FPS: 21.7308
Figure(640x480)
Avg FPS: 21.7312
Figure(640x480)
Avg FPS: 21.7312
Figure(640x480)
Avg FPS: 21.7307
Figure(640x480)
Avg FPS: 21.7307
Figure(640x480)
Avg FPS: 21.7307
Figure(640x480)
Avg FPS: 21.7316
Figure(640x480)
Avg FPS: 21.7317
Figure(640x480)
Avg FPS: 21.7309
Figure(640x480)
Avg FPS: 21.7302
Figure(640x480)
Avg FPS: 21.7310
Figure(640x480)
Avg FPS: 21.7317
Figure(640x480)
Avg FPS: 21.7313
Figure(640x480)
Avg FPS: 21.7314
Figure(6

In [23]:
# Run just the raw model on the first 1k images of the validation set
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --benchmark --max_images=1000

Config not specified. Parsed yolact_base_config from the file name.

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1275: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)
loading annotations into memory...
Done (t=0.74s)
creating index...
index created!
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'lat_l

In [24]:
# Display qualitative results on the specified image.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.3 --top_k=100 --images=/content/test_images/6946559805_7edf757660_z.jpg:output_image_1.jpg

# Process an image and save it to another file.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.3 --top_k=100 --images=input_image.png:output_image.png

# Process a whole folder of images.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.3 --top_k=100 --images=/content/output_images:/content/output_images

Config not specified. Parsed yolact_base_config from the file name.

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1275: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Co

In [25]:
!pwd

# Display a video in real-time. "--video_multiframe" will process that many frames at once for improved performance.
# Note: Real-time display for videos might not work directly in Colab without specific display server setup.
# It's more reliable to save the processed video to a file.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.3 --top_k=100 --video_multiframe=2 --video=/content/myvideo.mp4:/content/output_video_processed.mp4

# Display a webcam feed in real-time. If you have multiple webcams pass the index of the webcam you want instead of 0.
# This command will not work in a typical Colab environment as it doesn't have access to your local webcam.
# !python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.3 --top_k=100 --video_multiframe=2 --video=0

# Process a video and save it to another file.
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.3 --top_k=100 --video=/content/input_video.mp4:/content/output_video.mp4

/content/yolact
Config not specified. Parsed yolact_base_config from the file name.

/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1275: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'lat_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'downsample_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-consta

In [26]:
from google.colab import files

print("Please upload your file(s):")
uploaded = files.upload()

for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')


Please upload your file(s):


Saving myvideo.mp4 to myvideo.mp4
User uploaded file "myvideo.mp4" with length 6541327 bytes


After uploading, the files will be available in the current working directory. If you uploaded a video file, you can then try running the `eval.py` command for video processing again, making sure the video file name matches what you uploaded.

In [ ]:
# Trains using the base config with a batch size of 8 (the default).
!python train.py --config=yolact_base_config

# Trains yolact_base_config with a batch_size of 5. For the 550px models, 1 batch takes up around 1.5 gigs of VRAM, so specify accordingly.
!python train.py --config=yolact_base_config --batch_size=5

# Resume training yolact_base with a specific weight file and start from the iteration specified in the weight file's name.
!python train.py --config=yolact_base_config --resume=weights/yolact_base_10_32100.pth --start_iter=-1

# Use the help option to see a description of all available command line arguments
!python train.py --help

/content/yolact/utils/nvinfo.py:13: SyntaxWarning: invalid escape sequence '\('
  gpu_infos = [re.match('GPU ([0-9]+): ([^(]+) \(UUID: ([^)]+)\)', gpu).groups() for gpu in gpus]
/usr/local/lib/python3.12/dist-packages/torch/__init__.py:1275: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)
loading annotations into memory...
Done (t=14.92s)
creating index...
index created!
loading annotations into memory...
Done (t=0.53s)
creating index...
index created!
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'pred_layers' was found in ScriptModule constants,  but it is a non-constant submodule. Consider removing it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/jit/_recursive.py:313: UserWarning: 'downsample_layers' was foun

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

# Correctly point to the output directory used by eval.py
output_images = Path('/content/yolact/output_images_dir')

def show_image(image_path):
  img = cv2.imread(image_path)
  # Check if image was loaded correctly
  if img is None:
      print(f"Warning: Could not load image from {image_path}")
      return
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.axis('off') # Hide axes ticks
  plt.show()

# Iterate through all of the output images and display them
if output_images.exists() and output_images.is_dir():
    for img_path in output_images.iterdir():
        print(f"Displaying: {img_path}")
        show_image(str(img_path))
else:
    print(f"The directory '{output_images}' does not exist or is not a directory. Cannot display images.")
    print("You might need to run the 'eval.py' command to generate output images first.")

In [ ]:
import os

# Ensure we are in the /content directory first
%cd /content

# Check if the yolact directory exists; if not, clone it
if not os.path.exists('yolact'):
    print("Yolact directory not found, attempting to clone again...")
    !git clone https://github.com/dbolya/yolact.git
    if not os.path.exists('yolact'):
        print("Error: Failed to clone yolact repository. Video processing cannot proceed.")
        # Exit or handle the error appropriately if cloning fails
    else:
        print("Yolact repository cloned successfully.")
else:
    print("Yolact directory already exists.")

# Change into the yolact directory
%cd yolact

# Re-install PyTorch and TorchVision with CUDA support to ensure the correct version is loaded
# This step is crucial if previous installations (e.g., from requirements.txt) downgraded PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Verify CUDA availability after re-installation
import torch
if not torch.cuda.is_available():
    print("Warning: CUDA is still not available after re-installation. Please ensure your Colab runtime type is set to GPU (Runtime -> Change runtime type).")
else:
    print(f"CUDA available: {torch.cuda.is_available()}")

# Process the uploaded video and save the output
# This command assumes 'myvideo.mp4' is located in /content (one level up from yolact)
!python eval.py --trained_model=weights/yolact_base_54_800000.pth --score_threshold=0.3 --top_k=100 --video_multiframe=2 --video=/content/myvideo.mp4:/content/output_video_processed.mp4

# Display the processed video in the notebook
from IPython.display import Video
Video("/content/output_video_processed.mp4", embed=True)